In [1]:
## IMPORTACIÓN GENERAL DE LIBRERIAS Y VISUALIZACIÓN DE DATOS (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as DT
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.cross_validation import train_test_split
import xgboost as xgb

%matplotlib inline
warnings.filterwarnings('ignore')
plt.style.use('default') 
sns.set(style="whitegrid") 
plt.rcParams['figure.figsize'] = (15, 10)
pd.set_option('display.max_columns', 1000)
np.set_printoptions(threshold=np.nan)

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
################################################################
## EVENTS.
################################################################
## OBTENEMOS TODA LA INFORMACIÓN DE LOS DIFERENTES CSV.
events = pd.read_csv('../../events.csv')
## TRANSFORMACIÓN DE TIPOS PARA EL CSV (EVENTS)
events['wifi'] = events['wifi'].fillna(False).astype(bool)
events['connection_type'] = events['connection_type'].astype('category')
events['trans_id'] = events['trans_id'].astype('category')
events['date'] = pd.to_datetime(events['date'], infer_datetime_format=True)
## DIFERENCIAMOS EN TRES COLUMNAS DIFERENTES EL DIA, MES Y AÑO.
events['mes'] = events['date'].dt.month
events['dia'] = events['date'].dt.day
events['hora'] = events['date'].dt.hour
# ARMAMOS UNA LÓGICA PARA SEGMENTAR LAS FRANJAS HORARIAS.
# MADRUGADA de 00 a 06
events['hora_madrugada'] = 0
events.loc[((events.hora > -1) & (events.hora < 7)), 'hora_madrugada'] = 1
# MAÑANA de 07 a 11
events['hora_maniana'] = 0
events.loc[((events.hora > 6) & (events.hora < 12)), 'hora_maniana'] = 1
# ALMUERZO de 12 a 13
events['hora_almuerzo'] = 0
events.loc[((events.hora > 11) & (events.hora < 14)), 'hora_almuerzo'] = 1
# TARDE de 14 a 18
events['hora_tarde'] = 0
events.loc[((events.hora > 13) & (events.hora < 19)), 'hora_tarde'] = 1
# NOCHE de 19 a 23
events['hora_noche'] = 0
events.loc[((events.hora > 18) & (events.hora < 24)), 'hora_noche'] = 1
################################################################
################################################################

In [3]:
##################### connection_type
dummies = pd.get_dummies(events['connection_type'], drop_first=False)
events = pd.concat([events, dummies], axis=1)
del events['connection_type']

In [4]:
##################### event_uuid
del events['event_uuid']

In [5]:
##################### date
del events['date']

In [6]:
##################### wifi
events['wifi_value'] = 2
events.loc[events.wifi == False, 'wifi_value'] = 0
events.loc[events.wifi == True, 'wifi_value'] = 1
del events['wifi']

In [7]:
##################### attributed
events['attributed_value'] = 2
events.loc[events.attributed == False, 'attributed_value'] = 0
events.loc[events.attributed == True, 'attributed_value'] = 1
del events['attributed']

In [15]:
##################### trans_id
events['trans_id_value'] = events['trans_id'].cat.codes
events.loc[events.trans_id_value == -1, 'trans_id_value'] = 0
del events['trans_id']

In [18]:
# trans_id category
# events['ref_hash'].value_counts()
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7744581 entries, 0 to 7744580
Data columns (total 32 columns):
index                 int64
event_id              int64
ref_type              int64
ref_hash              int64
application_id        int64
device_countrycode    int64
device_os_version     float64
device_brand          float64
device_model          float64
device_city           float64
session_user_agent    float64
user_agent            float64
carrier               float64
kind                  float64
device_os             float64
ip_address            int64
device_language       float64
mes                   int64
dia                   int64
hora                  int64
hora_madrugada        int64
hora_maniana          int64
hora_almuerzo         int64
hora_tarde            int64
hora_noche            int64
Cable/DSL             uint8
Cellular              uint8
Corporate             uint8
Dialup                uint8
wifi_value            int64
attributed_value      int64

In [19]:
################################################################
## Ventana 1: Del 18 al 20
## Ventana 2: Del 19 al 21
## Ventana 3: Del 20 al 22
## Ventana 4: Del 21 al 23
## Ventana 5: Del 22 al 24
################################################################
events_Ventana1 = events[(events['dia'] >= 18) & (events['dia'] <= 20)]
## events_Ventana2 = events[(events['dia'] >= 19) & (events['dia'] <= 21)]
## events_Ventana3 = events[(events['dia'] >= 20) & (events['dia'] <= 22)]
## events_Ventana4 = events[(events['dia'] >= 21) & (events['dia'] <= 23)]
## events_Ventana5 = events[(events['dia'] >= 22) & (events['dia'] <= 24)]
events = ""
################################################################

In [35]:
################################################################
## CLICKS.
################################################################
## OBTENEMOS TODA LA INFORMACIÓN DE LOS DIFERENTES CSV.
clicks = pd.read_csv('../../clicks.csv')
## TRANSFORMACIÓN DE TIPOS PARA EL CSV (CLICKS)
clicks['created'] = pd.to_datetime(clicks['created'], infer_datetime_format=True)
## DIFERENCIAMOS EN TRES COLUMNAS DIFERENTES EL DIA, MES Y AÑO.
clicks['mes'] = clicks['created'].dt.month
clicks['dia'] = clicks['created'].dt.day
clicks['hora'] = clicks['created'].dt.hour
# ARMAMOS UNA LÓGICA PARA SEGMENTAR LAS FRANJAS HORARIAS.
# MADRUGADA de 00 a 06
clicks['hora_madrugada'] = 0
clicks.loc[((clicks.hora > -1) & (clicks.hora < 7)), 'hora_madrugada'] = 1
# MAÑANA de 07 a 11
clicks['hora_maniana'] = 0
clicks.loc[((clicks.hora > 6) & (clicks.hora < 12)), 'hora_maniana'] = 1
# ALMUERZO de 12 a 13
clicks['hora_almuerzo'] = 0
clicks.loc[((clicks.hora > 11) & (clicks.hora < 14)), 'hora_almuerzo'] = 1
# TARDE de 14 a 18
clicks['hora_tarde'] = 0
clicks.loc[((clicks.hora > 13) & (clicks.hora < 19)), 'hora_tarde'] = 1
# NOCHE de 19 a 23
clicks['hora_noche'] = 0
clicks.loc[((clicks.hora > 18) & (clicks.hora < 24)), 'hora_noche'] = 1
################################################################
################################################################

In [37]:
clicks.fillna(0, inplace = True)
clicks['trans_id'] = clicks['trans_id'].astype('category')

In [40]:
##################### date
del clicks['created']

In [42]:
##################### wifi_connection
clicks['wifi_connection'] = 2
clicks.loc[clicks.wifi_connection == False, 'wifi_value'] = 0
clicks.loc[clicks.wifi_connection == True, 'wifi_value'] = 1
del clicks['wifi_connection']

In [43]:
##################### trans_id
clicks['trans_id_value'] = clicks['trans_id'].cat.codes
clicks.loc[clicks.trans_id_value == -1, 'trans_id_value'] = 0
del clicks['trans_id']

In [46]:
clicks.loc[clicks.touchX == 'Infinity', 'touchX'] = 2
clicks.loc[clicks.touchY == 'Infinity', 'touchY'] = 2

In [48]:
clicks['touchX'] = clicks['touchX'].astype(float).fillna(0.0)
clicks['touchY'] = clicks['touchY'].astype(float).fillna(0.0)

In [49]:
clicks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64296 entries, 0 to 64295
Data columns (total 27 columns):
advertiser_id     64296 non-null int64
action_id         64296 non-null float64
source_id         64296 non-null int64
country_code      64296 non-null int64
latitude          64296 non-null float64
longitude         64296 non-null float64
carrier_id        64296 non-null float64
os_minor          64296 non-null float64
agent_device      64296 non-null float64
os_major          64296 non-null float64
specs_brand       64296 non-null int64
brand             64296 non-null float64
timeToClick       64296 non-null float64
touchX            64296 non-null float64
touchY            64296 non-null float64
ref_type          64296 non-null int64
ref_hash          64296 non-null int64
mes               64296 non-null int64
dia               64296 non-null int64
hora              64296 non-null int64
hora_madrugada    64296 non-null int64
hora_maniana      64296 non-null int64
hora_almuer

In [50]:
################################################################
## Ventana 1: Del 18 al 20
## Ventana 2: Del 19 al 21
## Ventana 3: Del 20 al 22
## Ventana 4: Del 21 al 23
## Ventana 5: Del 22 al 24
################################################################
clicks_Ventana1 = clicks[(clicks['dia'] >= 18) & (clicks['dia'] <= 20)]
## clicks_Ventana2 = clicks[(clicks['dia'] >= 19) & (clicks['dia'] <= 21)]
## clicks_Ventana3 = clicks[(clicks['dia'] >= 20) & (clicks['dia'] <= 22)]
## clicks_Ventana4 = clicks[(clicks['dia'] >= 21) & (clicks['dia'] <= 23)]
## clicks_Ventana5 = clicks[(clicks['dia'] >= 22) & (clicks['dia'] <= 24)]
clicks = ""
################################################################

In [63]:
################################################################
## INSTALLS.
################################################################
## OBTENEMOS TODA LA INFORMACIÓN DE LOS DIFERENTES CSV.
installs = pd.read_csv('../../installs.csv')
## TRANSFORMACIÓN DE TIPOS PARA EL CSV (INSTALLS)
installs['kind'] = installs['kind'].astype('category')
installs['user_agent'] = installs['user_agent'].astype('category')
installs['session_user_agent'] = installs['session_user_agent'].astype('category')
installs['created'] = pd.to_datetime(installs['created'], infer_datetime_format=True)
installs['trans_id'] = installs['trans_id'].astype('category')
## DIFERENCIAMOS EN TRES COLUMNAS DIFERENTES EL DIA, MES Y AÑO.
installs['mes'] = installs['created'].dt.month
installs['dia'] = installs['created'].dt.day
installs['hora'] = installs['created'].dt.hour
# ARMAMOS UNA LÓGICA PARA SEGMENTAR LAS FRANJAS HORARIAS.
# MADRUGADA de 00 a 06
installs['hora_madrugada'] = 0
installs.loc[((installs.hora > -1) & (installs.hora < 7)), 'hora_madrugada'] = 1
# MAÑANA de 07 a 11
installs['hora_maniana'] = 0
installs.loc[((installs.hora > 6) & (installs.hora < 12)), 'hora_maniana'] = 1
# ALMUERZO de 12 a 13
installs['hora_almuerzo'] = 0
installs.loc[((installs.hora > 11) & (installs.hora < 14)), 'hora_almuerzo'] = 1
# TARDE de 14 a 18
installs['hora_tarde'] = 0
installs.loc[((installs.hora > 13) & (installs.hora < 19)), 'hora_tarde'] = 1
# NOCHE de 19 a 23
installs['hora_noche'] = 0
installs.loc[((installs.hora > 18) & (installs.hora < 24)), 'hora_noche'] = 1
################################################################
################################################################

In [64]:
installs['click_hash'].fillna(0, inplace = True)
installs['device_brand'].fillna(0, inplace = True)
installs['device_model'].fillna(0, inplace = True)
installs['device_language'].fillna(0, inplace = True)
del installs['event_uuid']

In [67]:
##################### wifi
installs['wifi_value'] = 2
installs.loc[installs.wifi == False, 'wifi_value'] = 0
installs.loc[installs.wifi == True, 'wifi_value'] = 1
del installs['wifi']

In [71]:
##################### attributed
installs['attributed_value'] = 2
installs.loc[installs.attributed == False, 'attributed_value'] = 0
installs.loc[installs.attributed == True, 'attributed_value'] = 1
del installs['attributed']

In [72]:
##################### implicit
installs['implicit_value'] = 2
installs.loc[installs.implicit == False, 'implicit_value'] = 0
installs.loc[installs.implicit == True, 'implicit_value'] = 1
del installs['implicit']

In [69]:
installs['click_hash'] = installs['click_hash'].astype('category')

In [75]:
installs['session_user_agent_value'] = installs['session_user_agent'].cat.codes
del installs['session_user_agent']

In [79]:
installs.loc[installs.session_user_agent_value == -1, 'session_user_agent_value'] = 0

4530    335311
4528     75354
6        15193
0        14840
2478      9204
4431      2018
11        2015
839        681
4422       504
1743       426
2210       307
4357       301
558        299
2117       285
851        281
2057       254
1984       248
878        228
2112       216
1933       212
1566       184
2027       171
1895       164
1894       158
1790       155
2240       146
2175       145
1749       140
1321       139
391        134
         ...  
356          1
4198         1
100          1
3003         1
3259         1
3515         1
1379         1
3426         1
1123         1
1380         1
442          1
4536         1
2660         1
1381         1
3000         1
953          1
3172         1
3256         1
2916         1
1209         1
4455         1
3685         1
3512         1
3768         1
4199         1
101          1
2148         1
4024         1
4280         1
4220         1
Name: session_user_agent_value, Length: 4537, dtype: int64

In [86]:
##################### click_hash
installs['click_hash_values'] = clicks['click_hash'].cat.codes
installs.loc[installs.click_hash_value == -1, 'click_hash_value'] = 0
del installs['click_hash']

TypeError: string indices must be integers, not str

In [76]:
installs.info()
#user_agent            330768 non-null category
#kind                  103168 non-null category
#trans_id              8933 non-null category
#click_hash            481511 non-null category

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481511 entries, 0 to 481510
Data columns (total 25 columns):
created                     481511 non-null datetime64[ns]
application_id              481511 non-null int64
ref_type                    481511 non-null int64
ref_hash                    481511 non-null int64
click_hash                  481511 non-null category
device_countrycode          481511 non-null int64
device_brand                481511 non-null float64
device_model                481511 non-null float64
user_agent                  330768 non-null category
kind                        103168 non-null category
trans_id                    8933 non-null category
ip_address                  481511 non-null int64
device_language             481511 non-null float64
mes                         481511 non-null int64
dia                         481511 non-null int64
hora                        481511 non-null int64
hora_madrugada              481511 non-null int64
hora_maniana  

In [18]:
################################################################
## Ventana 1: Del 18 al 20
## Ventana 2: Del 19 al 21
## Ventana 3: Del 20 al 22
## Ventana 4: Del 21 al 23
## Ventana 5: Del 22 al 24
################################################################
installs_Ventana1 = installs[(installs['dia'] >= 18) & (installs['dia'] <= 20)]
## installs_Ventana2 = installs[(installs['dia'] >= 19) & (installs['dia'] <= 21)]
## installs_Ventana3 = installs[(installs['dia'] >= 20) & (installs['dia'] <= 22)]
## installs_Ventana4 = installs[(installs['dia'] >= 21) & (installs['dia'] <= 23)]
## installs_Ventana5 = installs[(installs['dia'] >= 22) & (installs['dia'] <= 24)]
installs = ""
################################################################

In [5]:
################################################################
## AUCTIONS.
################################################################
## OBTENEMOS TODA LA INFORMACIÓN DE LOS DIFERENTES CSV.
auctions = pd.read_csv('../../auctions.csv',nrows=5000000)
## TRANSFORMACIÓN DE TIPOS PARA EL CSV (AUCTIONS)
auctions['date'] = pd.to_datetime(auctions['date'], infer_datetime_format=True)
## DIFERENCIAMOS EN TRES COLUMNAS DIFERENTES EL DIA, MES Y AÑO.
auctions['mes'] = auctions['date'].dt.month
auctions['dia'] = auctions['date'].dt.day
auctions['hora'] = auctions['date'].dt.hour
# ARMAMOS UNA LÓGICA PARA SEGMENTAR LAS FRANJAS HORARIAS.
# MADRUGADA de 00 a 06
auctions['hora_madrugada'] = 0
auctions.loc[((auctions.hora > -1) & (auctions.hora < 7)), 'hora_madrugada'] = 1
# MAÑANA de 07 a 11
auctions['hora_maniana'] = 0
auctions.loc[((auctions.hora > 6) & (auctions.hora < 12)), 'hora_maniana'] = 1
# ALMUERZO de 12 a 13
auctions['hora_almuerzo'] = 0
auctions.loc[((auctions.hora > 11) & (auctions.hora < 14)), 'hora_almuerzo'] = 1
# TARDE de 14 a 18
auctions['hora_tarde'] = 0
auctions.loc[((auctions.hora > 13) & (auctions.hora < 19)), 'hora_tarde'] = 1
# NOCHE de 19 a 23
auctions['hora_noche'] = 0
auctions.loc[((auctions.hora > 18) & (auctions.hora < 24)), 'hora_noche'] = 1
################################################################
################################################################

In [19]:
################################################################
## Ventana 1: Del 18 al 20
## Ventana 2: Del 19 al 21
## Ventana 3: Del 20 al 22
## Ventana 4: Del 21 al 23
## Ventana 5: Del 22 al 24
################################################################
auctions_Ventana1 = auctions[(auctions['dia'] >= 18) & (auctions['dia'] <= 20)]
## auctions_Ventana2 = auctions[(auctions['dia'] >= 19) & (auctions['dia'] <= 21)]
## auctions_Ventana3 = auctions[(auctions['dia'] >= 20) & (auctions['dia'] <= 22)]
## auctions_Ventana4 = auctions[(auctions['dia'] >= 21) & (auctions['dia'] <= 23)]
## auctions_Ventana5 = auctions[(auctions['dia'] >= 22) & (auctions['dia'] <= 24)]
auctions = ""
################################################################

In [13]:
################################################################
## TARGET.
################################################################
## OBTENEMOS TODA LA INFORMACIÓN DE LOS DIFERENTES CSV.
targets = pd.read_csv('../../target.csv')
################################################################
################################################################

In [8]:
events_Ventana1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2553458 entries, 1417 to 7744580
Data columns (total 31 columns):
index                 int64
date                  datetime64[ns]
event_id              int64
ref_type              int64
ref_hash              int64
application_id        int64
attributed            bool
device_countrycode    int64
device_os_version     float64
device_brand          float64
device_model          float64
device_city           float64
session_user_agent    float64
trans_id              category
user_agent            float64
event_uuid            object
carrier               float64
kind                  float64
device_os             float64
wifi                  bool
connection_type       category
ip_address            int64
device_language       float64
mes                   int64
dia                   int64
hora                  int64
hora_madrugada        int64
hora_maniana          int64
hora_almuerzo         int64
hora_tarde            int64
hora_noche

In [8]:
clicks_Ventana1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26351 entries, 0 to 26350
Data columns (total 28 columns):
advertiser_id      26351 non-null int64
action_id          0 non-null float64
source_id          26351 non-null int64
created            26351 non-null datetime64[ns]
country_code       26351 non-null int64
latitude           26351 non-null float64
longitude          26351 non-null float64
wifi_connection    26351 non-null bool
carrier_id         26340 non-null float64
trans_id           26351 non-null object
os_minor           26339 non-null float64
agent_device       3243 non-null float64
os_major           26339 non-null float64
specs_brand        26351 non-null int64
brand              6235 non-null float64
timeToClick        22977 non-null float64
touchX             23011 non-null float64
touchY             23011 non-null float64
ref_type           26351 non-null int64
ref_hash           26351 non-null int64
mes                26351 non-null int64
dia                26351 n

In [9]:
installs_Ventana1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3412 entries, 0 to 3411
Data columns (total 26 columns):
created               3412 non-null datetime64[ns]
application_id        3412 non-null int64
ref_type              3412 non-null int64
ref_hash              3412 non-null int64
click_hash            0 non-null float64
attributed            3412 non-null bool
implicit              3412 non-null bool
device_countrycode    3412 non-null int64
device_brand          1047 non-null float64
device_model          3411 non-null float64
session_user_agent    3364 non-null category
user_agent            1729 non-null category
event_uuid            865 non-null object
kind                  865 non-null category
wifi                  1729 non-null object
trans_id              6 non-null category
ip_address            3412 non-null int64
device_language       3378 non-null float64
mes                   3412 non-null int64
dia                   3412 non-null int64
hora                  3412 non-n

In [10]:
auctions_Ventana1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000000 entries, 0 to 4999999
Data columns (total 15 columns):
auction_type_id    float64
country            int64
date               datetime64[ns]
device_id          int64
platform           int64
ref_type_id        int64
source_id          int64
mes                int64
dia                int64
hora               int64
hora_madrugada     int64
hora_maniana       int64
hora_almuerzo      int64
hora_tarde         int64
hora_noche         int64
dtypes: datetime64[ns](1), float64(1), int64(13)
memory usage: 572.2 MB


In [11]:
targets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5930 entries, 0 to 5929
Data columns (total 2 columns):
ref_hash    5930 non-null object
obj         5930 non-null int64
dtypes: int64(1), object(1)
memory usage: 92.7+ KB
